# Instalaciones

In [1]:
!pip install --no-cache-dir transformers sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 70.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 311.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 229.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 227.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 311.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 226.6 MB/s eta 0:00:00


# imports

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
import pandas as pd
import re
from google.colab import drive
drive.mount('/content/drive')
!ls '/content/drive/MyDrive/TEC/Proyecto/Investigacion'

Mounted at /content/drive
 Alpaca						   Instrucciones_Plan_de_Proyecto.pdf
'Analisis cualitativo de seleccion.gdoc'	   Instrucciones_Requerimientos_de_Software.pdf
'Aplicacion Metricas completas.ipynb'		   Investigacion.ipynb
'BRENES BERTA - Resultado Ficha Proyecto.pdf'	   llama
 Comparacion_simplificación_sample.xlsx		   Llama
 Dataset5000					   manually_utf8.csv
 dataset_replicas2.csv				   Mavin_A_Rolls_Royce_EARS_RE09_Paperaccepted.pdf
 dataset_replicas3.csv				   Metrics_sari_bleu.ipynb
 dataset_replicas4.csv				   Pegasus.ipynb
 dataset_replicas5.csv				  'Plan de proyecto.gdoc'
'Documento de Diseño-Bertha Brenes.gdoc'	   Poster.gdoc
'Documento de requerimiento.gdoc'		  'Probando el dataset.ipynb'
'Documento para el profe.gdoc'			  'Recoleccion de datos.gdoc'
 FastChat.ipynb					   salidaPegasus2.csv
'Ficha de presentación de CE-5302 BerthaBB.docx'   sampleLlama2.csv
 FinalRep.csv					   samples
 Financial_Dataset_in_Spanish.pdf		  'Tarea 2.gdoc'
'Hoja de cálculo sin tít

# Cargar el modelo

In [3]:
# Load model directly
tokenizer = AutoTokenizer.from_pretrained("lmsys/fastchat-t5-3b-v1.0")
model = AutoModelForSeq2SeqLM.from_pretrained("lmsys/fastchat-t5-3b-v1.0")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# Aplicar el modelo distinto de pipeline

In [ ]:
def aplicacionModelo(input):
  inputs = tokenizer(input, return_tensors="pt").to('cpu') #puedes cambiar a 'cuda' si tienes GPU
  outputs = model.generate(inputs["input_ids"],
                          do_sample = True,
                          max_length = 256, #puedes subir este parametro hasta 500
                          num_return_sequences=2, #recomiendo hasta 6 para que no demore mucho
                          top_k=50,
                          top_p=0.90,
                          )
  detok_outputs = [tokenizer.decode(x, skip_special_tokens=True) for x in outputs]

  for output in detok_outputs:
      print(output)
      print("\n") # imprime un salto de linea para separar cada uno de los outputs (en el caso que num_return_sequences sea mayor que 1)

In [ ]:
input_text2_simpli ="""instrut5: Podrias simplificar esta frase: A pesar de los esfuerzos y la mejora de los laboratorios, es imposible que se elabore una solución adecuada y a tiempo para algunos códigos malignos que se propagan en cuestión de minutos."""
print('input', input_text2_simpli)
aplicacionModelo(input_text2_simpli)

input instrut5: Podrias simplificar esta frase: A pesar de los esfuerzos y la mejora de los laboratorios, es imposible que se elabore una solución adecuada y a tiempo para algunos códigos malignos que se propagan en cuestión de minutos.


NameError: ignored

# Aplicar el modelo con pipeline

In [ ]:
generator = pipeline(task="summarization", model=model, tokenizer=tokenizer)
input_text2_simpli ="""instrut5: Podrias simplificar esta frase: A pesar de los esfuerzos y la mejora de los laboratorios, es imposible que se elabore una solución adecuada y a tiempo para algunos códigos malignos que se propagan en cuestión de minutos."""

generadorRes =  generator(input_text2_simpli, max_length=73)
print(generadorRes[0]['summary_text'])

<pad> Despite  the  efforts  and  improvements  in  labs,  it  is  impossible  to  develop  a  suitable  solution  for  some  malicious  code  that  spreads  quickly.



In [4]:
generator = pipeline( task= "text-generation",model=model, tokenizer=tokenizer)
input_text2_simpli ="""A pesar de los esfuerzos y la mejora de los laboratorios, es imposible que se elabore una solución adecuada y a tiempo para algunos códigos malignos que se propagan en cuestión de minutos."""
prompt = """<s>[INST] Podrias simplificar la siguiente frase? "A pesar de los esfuerzos y la mejora de los laboratorios, es imposible que se elabore una solución adecuada y a tiempo para algunos códigos malignos que se propagan en cuestión de minutos." [/INST] """
prompt2 = """instrut5: <in>Podrias simplificar la siguiente frase?<in> "A pesar de los esfuerzos y la mejora de los laboratorios, es imposible que se elabore una solución adecuada y a tiempo para algunos códigos malignos que se propagan en cuestión de minutos" """
generadorRes =  generator(prompt, max_length=33)
print(generadorRes)
print( generator(prompt2, max_length=33))

The model 'T5ForConditionalGeneration' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonForCausalLM', 'PLBartForCausalLM', '

[{'generated_text': '<s>[INST] Podrias simplificar la siguiente frase? "A pesar de los esfuerzos y la mejora de los laboratorios, es imposible que se elabore una solución adecuada y a tiempo para algunos códigos malignos que se propagan en cuestión de minutos." [/INST] '}]
[{'generated_text': 'instrut5: <in>Podrias simplificar la siguiente frase?<in> "A pesar de los esfuerzos y la mejora de los laboratorios, es imposible que se elabore una solución adecuada y a tiempo para algunos códigos malignos que se propagan en cuestión de minutos" '}]


In [5]:
def applyPipelineSummary(value):
  print(value)
  generadorRes =  generator(value, max_length=65)
  texto_limpio = re.sub(r'<pad>', '', generadorRes[0]['summary_text'])
  return texto_limpio

def applyPipeline(value):
  valueIns = "instrut5: <in>Podrias simplificar la siguiente frase?<in>"+value
  print(valueIns)
  generadorRes =  generator(valueIns, max_length= 20)
  texto_limpio = re.sub(r'instrut5:|<in>.*?<in>', '', generadorRes[0]['generated_text'])
  return texto_limpio

# Aplicacion del modelo a los dataset

## Manually

In [ ]:
path_file_dataset_man = "/content/drive/MyDrive/TEC/Proyecto/Investigacion/manually_utf8.csv"
df = pd.read_csv(path_file_dataset_man, nrows=100)

df['simpleFastChat'] = df['complex'].apply(applyPipeline)

df.to_csv("/content/drive/MyDrive/TEC/Proyecto/Investigacion/Dataset5000/salidaFastC.csv", index=False)


## FinalRep

In [ ]:
path_file_dataset_man = "/content/drive/MyDrive/TEC/Proyecto/Investigacion/FinalRep.csv"
df = pd.read_csv(path_file_dataset_man, )

df['simple'] = df['Segmento'].apply(applyPipeline)

df.to_csv("/content/drive/MyDrive/TEC/Proyecto/Investigacion/Dataset5000/salidaFastC_FinalRep.csv", index=False)

instrut5: <in>Podrias simplificar la siguiente frase?<in>Creer que por tener estos objetos seremos más fel ices sólo por “estar a la moda”, que seremos aceptados dentro de un círculo particular, que tendremos más amigos, es una ilusión consumista que sólo p roducirá más vacío y problemas personales.
instrut5: <in>Podrias simplificar la siguiente frase?<in>Este recipiente tiene el nombre de gastos de emergencia, y su contenido debería ser necesario para cubrir al menos 6 meses de gastos sin ingresos (o en el caso óptimo, 1 año).
instrut5: <in>Podrias simplificar la siguiente frase?<in>Invertir en opciones permite realizar estrategias especulativas así como tomar posiciones de cobertura frente a los movimientos de mercado.
instrut5: <in>Podrias simplificar la siguiente frase?<in>Futuros: un futuro es un contrato por el cual las partes se comprometen a comprar o vender un determinado activo subyacente a un precio ya fijado, y a una fecha también preestablecida.
instrut5: <in>Podrias simpl

# Aplicacion del modello al sample de 13 datos

In [ ]:
path_file_dataset_man = "/content/drive/MyDrive/TEC/Proyecto/Investigacion/samples/sampleFastChat2.csv"
df = pd.read_csv(path_file_dataset_man,)

df['simple2'] = df['complex'].apply(applyPipeline)
print(df['simple2'])

df.to_csv("/content/drive/MyDrive/TEC/Proyecto/Investigacion/samples/sampleFastChat2.csv", index=False)

instrut5: <in>Podrias simplificar la siguiente frase?<in>El no recibir educación en tan sustantivos aspectos, además de que pone en duda la educación integral que tanto se pregona, ocasiona que las personas vivan y tengan que responder, después de recibir su educación, con un contundente analfabetismo económico y financiero en un entorno que, además de otras características, lo es también económico y financiero.Podrias simplificar esa frase?


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1285: UserWarning: Unfeasible length constraints: `min_length` (30) is larger than the maximum possible length (23). Generation will stop at the defined maximum length. You should decrease the minimum length and/or increase the maximum length.
  warnings.warn(


instrut5: <in>Podrias simplificar la siguiente frase?<in>Cuando se intenta definir el dinero, las personas piensan en monedas o billetes, lo cual es muy reducido porque el dinero es más que eso.
instrut5: <in>Podrias simplificar la siguiente frase?<in>A fin de mes, podrías tener unos $50 para depositar en tu cuenta de ahorro, que suman $600 al año (sin incluir los intereses, si los depositaras en una cuenta que pague intereses).
instrut5: <in>Podrias simplificar la siguiente frase?<in>Ingresos de tipo residual: Son ingresos producto de rendimientos financieros, arriendos de bienes raíces, dividendos, acciones de empresas; es decir, las personas no tienen que realizar un trabajo para recibir su ingreso.
instrut5: <in>Podrias simplificar la siguiente frase?<in>Este proceso permite a una empresa ceder el derecho de cobro de facturas o créditos comerciales a un inversionista a cambio de la obtención de un valor líquido que implica una rentabilidad con respecto al valor estipulado en el doc